In [5]:
import cv2
import dlib
import numpy as np
from google.colab import drive
from google.colab.patches import cv2_imshow
import os

# Mount Google Drive
drive.mount('/content/drive')

# Load Dlib's pre-trained face detector and shape predictor model
detector = dlib.get_frontal_face_detector()
predictor_path = '/content/drive/MyDrive/HeadPose/shape_predictor_68_face_landmarks.dat'

if not os.path.isfile(predictor_path):
    raise FileNotFoundError(f"Predictor file not found at {predictor_path}")

predictor = dlib.shape_predictor(predictor_path)

# Camera matrix and distortion coefficients (assuming no lens distortion)
cam_matrix = np.array([[640, 0, 320],
                       [0, 640, 240],
                       [0, 0, 1]], dtype='double')
dist_coeffs = np.zeros((4, 1))

# 3D model points of a generic face
model_points = np.array([
    (0.0, 0.0, 0.0),  # Nose tip
    (0.0, -330.0, -65.0),  # Chin
    (-225.0, 170.0, -135.0),  # Left eye left corner
    (225.0, 170.0, -135.0),  # Right eye right corner
    (-150.0, -150.0, -125.0),  # Left Mouth corner
    (150.0, -150.0, -125.0)  # Right mouth corner
])

def get_2d_image_points(shape):
    """ Get 2D image points from the 68 facial landmarks """
    image_points = np.array([
        (shape.part(30).x, shape.part(30).y),  # Nose tip
        (shape.part(8).x, shape.part(8).y),  # Chin
        (shape.part(36).x, shape.part(36).y),  # Left eye left corner
        (shape.part(45).x, shape.part(45).y),  # Right eye right corner
        (shape.part(48).x, shape.part(48).y),  # Left Mouth corner
        (shape.part(54).x, shape.part(54).y)  # Right mouth corner
    ], dtype='double')
    return image_points

def main(vid):
    cap = cv2.VideoCapture(vid)  # Capture video from the provided file
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/content/drive/MyDrive/HeadPose/final_video.mp4', fourcc, 25.0, (640, 360))


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            shape = predictor(gray, face)
            image_points = get_2d_image_points(shape)

            # SolvePnP to estimate head pose
            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, cam_matrix, dist_coeffs)

            # Project the points back to the 2D image plane
            (nose_end_point2D, _) = cv2.projectPoints(
                np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, cam_matrix, dist_coeffs)

            # Draw detected face landmarks
            for (x, y) in image_points:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), -1)

            # Draw a line from the nose tip to the projected point to indicate head direction
            p1 = (int(image_points[0][0]), int(image_points[0][1]))
            p2 = (int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
            cv2.line(frame, p1, p2, (255, 0, 0), 2)
            direction = "Head Direction: "

            if p2[0] > p1[0] + 20:
                direction += "Facing Right"
            elif p2[0] < p1[0] - 20:
                direction += "Facing Left"
            else:
                direction += "Facing Forward"

            #cv2.putText(frame, direction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 150, 50), 2, cv2.LINE_AA)
            cv2.putText(frame, direction, (face.left(), face.top() - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        out.write(frame)  # Write the frame to the video file

        # If you want to display the frame in Google Colab (optional)
        cv2_imshow(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    video_path = '/content/drive/MyDrive/HeadPose/genderdetection_video.mp4'  # Replace with your video file path
    main(video_path)

Output hidden; open in https://colab.research.google.com to view.